Weil ich noch nicht ganz sicher bin, was die besseren Ergebnisse bringt bei der Netzwerkanalyse, mache ich hier eine Version, die auch jene Paare erfasst, die von der gleichen Partei sind.

In [74]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
            
        geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
        regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
        datum = re.findall(regex, str(soup))[0]
                
        mini_dict = {
            "Datum" : datum,
            "Geschäft" : geschaeft_nr}
        df_list.append(mini_dict)

100%|██████████| 7026/7026 [04:35<00:00, 25.52it/s]


In [155]:
#Jetzt ist die Liste dreimal so lang! (Also: zwei Drittel aller Doppeleingaben sind nicht parteiübergreifend!)
len(df_list)

7025

In [156]:
df = pd.DataFrame(df_list)
df.head(5)

,Datum,Geschäft
0,27.6.2001,2001/362
1,5.4.2000,2000/171
2,12.9.2007,2007/496
3,14.03.2012,2012/99
4,27.10.2004,2004/555


bei diesen Positionen stimmt beim Datum etwas Grundsätzliches nicht, weshalb wir es mit einem annäherungsweisen Datum  aufgrund der Geschäftnummer ersetzen:

In [157]:
fehlerliste = ["29.01.20","08.01.20","18.12.19"]

for fehler in fehlerliste:
    fehlerposition = df[df["Datum"] == fehler].index.tolist()
    for zeile in fehlerposition:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        Ersatzdatum = "31.12." + Jahr
        df.loc[zeile, "Datum"] = df.loc[zeile, "Datum"].replace(fehler, Ersatzdatum)

Hier korrigieren wir zwei weitere Fehler, bei denen die Daten zwar stimmen, aber falsch formatiert sind (Leerschläge, zweistellige Jahreszahlen):

In [158]:
df_laenge = len(df)
for zeile in range(0, df_laenge-1):
    
    #hier holen wir jene Daten raus, die nur eine zweistellige als Jahreszahl haben und vervollständigen sie:
    if df.loc[zeile,"Datum"][-3:-2] == ".":
        if df.loc[zeile,"Datum"][-2:] >= "80":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"19"+df.loc[zeile,"Datum"][-2:]
        if df.loc[zeile,"Datum"][-2:] <= "21":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"20"+df.loc[zeile,"Datum"][-2:]
        
        print(df.loc[zeile,"Datum"])    
    
    #hier holen wir jene Daten raus, die einen Leerschlag drin haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-3:-2] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-3]+df.loc[zeile,"Datum"][-2:]
        
    #hier holen wir jene Daten mit vierer-Jahreszahlen raus, die einen Leerschlag davor haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-5:-4] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-5]+df.loc[zeile,"Datum"][-4:]
    
    #Hier holen wir jene raus, die zu kurz sind, weil die Jahreszahl ganz fehlt:
    if len(df.loc[zeile,"Datum"]) == 6:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"]+Jahr
        
        print(df.loc[zeile,"Datum"])

30.1.1991
29.3.1996
26.3.1993
18.3.1992
17.6.1992
22.10.1997
8.2.1995
9.11.1994
15.4.1992
17.09.2011
5.9.2005
15.5.1991


Heureka, jetzt sind alle Fehler draussen. Jetzt verwandlen wir die Daten ins Datetime-Format: 

In [159]:
df["Datum"] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')

In [160]:
df = df.set_index("Datum")
df.head(5)

,Geschäft
Datum,
2001-06-27,2001/362
2000-04-05,2000/171
2007-09-12,2007/496
2012-03-14,2012/99
2004-10-27,2004/555


Jetzt messen wir für die Statistik die Zahl pro Legislatur:

In [161]:
df_1990 = df["1990-05-15":"1994-05-14"]
df_1994 = df["1994-05-15":"1998-05-14"]
df_1998 = df["1998-05-15":"2002-05-14"]
df_2002 = df["2002-05-15":"2006-05-14"]
df_2006 = df["2006-05-15":"2010-05-14"]
df_2010 = df["2010-05-15":"2014-05-14"]
df_2014 = df["2014-05-15":"2018-05-14"]
df_2018 = df["2018-05-15":"2022-05-14"]

print("1990-94: "+str(len(df_1990)))
print("1994-98: "+str(len(df_1994)))
print("1998-02: "+str(len(df_1998)))
print("2002-06: "+str(len(df_2002)))
print("2006-10: "+str(len(df_2006)))
print("2010-14: "+str(len(df_2010)))
print("2014-18: "+str(len(df_2014)))
print("2018-22: "+str(len(df_2018)))

1990-94: 18
1994-98: 179
1998-02: 1362
2002-06: 1386
2006-10: 1427
2010-14: 1106
2014-18: 981
2018-22: 561
